# Ejercicio 1 — Árbol del Movimiento (Jerarquías y Transformaciones)

**Meta:** Comprender relaciones padre-hijo en escenas 3D y efectos de transformaciones acumuladas.

## Objetivos de Aprendizaje

1. **Jerarquías de Transformación**: Entender cómo las transformaciones se propagan de padres a hijos
2. **Transformaciones Acumuladas**: Visualizar el efecto cascada de rotaciones y traslaciones
3. **Espacios de Coordenadas**: Distinguir entre espacio local y espacio mundial

## Contexto Teórico

En gráficos por computadora 3D, los objetos se organizan en **grafos de escena** (scene graphs) donde:

- **Nodo Padre**: Aplica transformaciones que afectan a todos sus descendientes
- **Nodo Hijo**: Hereda transformaciones del padre y aplica las propias en su espacio local
- **Transformaciones Encadenadas**: Se multiplican matrices de transformación de forma recursiva

$$
M_{mundo} = M_{padre} \times M_{hijo} \times M_{nieto}
$$

### Ejemplo: Sistema Solar

```
Sol (rota sobre sí mismo)
└── Planeta (orbita al Sol + rota sobre sí mismo)
    └── Luna (orbita al Planeta + rota sobre sí mismo)
```

Cuando el Sol rota, arrastra consigo todo el sistema. Cuando el Planeta orbita, la Luna lo sigue.

## Instalación de Dependencias

Utilizaremos `pythreejs` para integrar Three.js en Jupyter.

In [1]:
# Instalación de dependencias
# Usar sys.executable asegura que instale en el Python correcto del kernel
import sys
!{sys.executable} -m pip install pythreejs ipywidgets numpy matplotlib Pillow imageio -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Habilitar extensiones de widgets
# Nota: Si usas JupyterLab, las extensiones se manejan automáticamente
import sys

# Intentar habilitar extensiones (funciona en Jupyter Notebook clásico)
try:
    !{sys.executable} -m jupyter nbextension enable --py --sys-prefix pythreejs 2>/dev/null
    !{sys.executable} -m jupyter nbextension enable --py --sys-prefix widgetsnbextension 2>/dev/null
    print("✅ Extensiones habilitadas (Jupyter Notebook)")
except:
    print("ℹ️  Usando JupyterLab - las extensiones se cargan automáticamente")

# Para JupyterLab, las extensiones se instalan con:
# jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-threejs
print("ℹ️  Si usas JupyterLab y los widgets no funcionan, ejecuta en terminal:")
print("   jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-threejs")

✅ Extensiones habilitadas (Jupyter Notebook)
ℹ️  Si usas JupyterLab y los widgets no funcionan, ejecuta en terminal:
   jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-threejs


## Importación de Bibliotecas

In [3]:
# Bibliotecas principales
import pythreejs as three
from IPython.display import display, HTML, Image
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, VBox, HBox, Label
import numpy as np
import time
from pathlib import Path

# Para captura de GIFs
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import imageio

print("✅ Bibliotecas importadas correctamente")
# Verificar que pythreejs se importó correctamente
try:
    print(f"📦 pythreejs instalado: {three.Group is not None}")
except:
    print("⚠️  pythreejs puede tener problemas de importación")

✅ Bibliotecas importadas correctamente
📦 pythreejs instalado: True


## Arquitectura de la Solución

### Patrón de Diseño: Composite Pattern

Implementaremos una jerarquía de 3 niveles usando el patrón **Composite**:

```
SceneNode (Abstract)
├── TransformNode (Concrete)
└── MeshNode (Concrete)
```

### Jerarquía de Transformaciones

```
Nivel 1: Cubo Padre (Rojo)
    - Rotación en Y (eje vertical)
    - Posición central
    └── Nivel 2: Cubo Hijo (Verde)
        - Orbita alrededor del padre
        - Rotación propia en X
        └── Nivel 3: Cubo Nieto (Azul)
            - Orbita alrededor del hijo
            - Rotación propia en Z
```

## Clase Base: Gestor de Jerarquías 3D

Aplicando principios **SOLID**:
- **S**ingle Responsibility: Cada clase tiene una única responsabilidad
- **O**pen/Closed: Extensible sin modificar código existente
- **L**iskov Substitution: Los nodos hijo pueden sustituir a los padres
- **I**nterface Segregation: Interfaces específicas para cada necesidad
- **D**ependency Inversion: Depende de abstracciones, no de concreciones

In [4]:
class HierarchicalNode:
    """
    Clase base para nodos jerárquicos en el grafo de escena.
    
    Implementa el patrón Composite para manejar transformaciones
    jerárquicas en Three.js.
    
    Attributes:
        name (str): Identificador del nodo
        group (three.Group): Grupo de Three.js que contiene la geometría
        children (list): Lista de nodos hijos
        local_transform (dict): Transformaciones locales (position, rotation, scale)
    """
    
    def __init__(self, name: str):
        self.name = name
        self.group = three.Group()
        self.children = []
        self.local_transform = {
            'position': [0, 0, 0],
            'rotation': [0, 0, 0],
            'scale': [1, 1, 1]
        }
    
    def add_child(self, child_node):
        """Agrega un nodo hijo y lo añade al grupo de Three.js"""
        self.children.append(child_node)
        self.group.add(child_node.group)
        return self
    
    def set_position(self, x: float, y: float, z: float):
        """Establece la posición local del nodo"""
        self.local_transform['position'] = [x, y, z]
        self.group.position = [x, y, z]
        return self
    
    def set_rotation(self, x: float, y: float, z: float):
        """Establece la rotación local del nodo (en radianes)"""
        self.local_transform['rotation'] = [x, y, z]
        # pythreejs requiere crear un objeto Euler explícitamente
        self.group.rotation = three.Euler(x, y, z, 'XYZ')
        return self
    
    def rotate(self, dx: float = 0, dy: float = 0, dz: float = 0):
        """Aplica rotación incremental"""
        current = self.local_transform['rotation']
        new_rotation = [
            current[0] + dx,
            current[1] + dy,
            current[2] + dz
        ]
        return self.set_rotation(*new_rotation)
    
    def get_three_object(self):
        """Retorna el objeto Three.js subyacente"""
        return self.group


class MeshNode(HierarchicalNode):
    """
    Nodo que contiene una malla 3D (geometría + material).
    
    Extiende HierarchicalNode agregando capacidades de renderizado.
    """
    
    def __init__(self, name: str, geometry, material):
        super().__init__(name)
        self.mesh = three.Mesh(
            geometry=geometry,
            material=material
        )
        self.group.add(self.mesh)
    
    def set_material_color(self, color: str):
        """Cambia el color del material"""
        self.mesh.material.color = color
        return self


print("✅ Clases base creadas correctamente")

✅ Clases base creadas correctamente


## Factory Pattern: Constructor de Nodos

Aplicamos el patrón **Factory** para encapsular la creación de objetos complejos.

In [5]:
class NodeFactory:
    """
    Factoría para crear nodos 3D con configuraciones predefinidas.
    
    Implementa el patrón Factory Method para encapsular la lógica
    de creación de objetos complejos.
    """
    
    @staticmethod
    def create_cube(name: str, size: float, color: str, wireframe: bool = False):
        """
        Crea un nodo con geometría de cubo.
        
        Args:
            name: Identificador del nodo
            size: Tamaño del cubo
            color: Color en formato hexadecimal
            wireframe: Si True, muestra solo bordes
            
        Returns:
            MeshNode configurado
        """
        geometry = three.BoxGeometry(size, size, size)
        material = three.MeshLambertMaterial(
            color=color,
            wireframe=wireframe,
            transparent=True,
            opacity=0.9
        )
        return MeshNode(name, geometry, material)
    
    @staticmethod
    def create_sphere(name: str, radius: float, color: str):
        """Crea un nodo con geometría esférica"""
        geometry = three.SphereGeometry(radius, 32, 32)
        material = three.MeshPhongMaterial(
            color=color,
            shininess=100
        )
        return MeshNode(name, geometry, material)
    
    @staticmethod
    def create_axes_helper(size: float = 5):
        """Crea un helper de ejes para referencia espacial"""
        return three.AxesHelper(size)


print("✅ NodeFactory creado correctamente")

✅ NodeFactory creado correctamente


## Construcción de la Jerarquía de 3 Niveles

Creamos el árbol de transformaciones con tres niveles de profundidad.

In [6]:
# Nivel 1: Nodo Padre (Cubo Rojo)
parent_node = NodeFactory.create_cube(
    name="Parent",
    size=2.0,
    color="#ff0000",  # Rojo
    wireframe=False
)
parent_node.set_position(0, 0, 0)

# Nivel 2: Nodo Hijo (Cubo Verde)
child_node = NodeFactory.create_cube(
    name="Child",
    size=1.5,
    color="#00ff00",  # Verde
    wireframe=False
)
child_node.set_position(4, 0, 0)  # Offset respecto al padre

# Nivel 3: Nodo Nieto (Cubo Azul)
grandchild_node = NodeFactory.create_cube(
    name="Grandchild",
    size=1.0,
    color="#0000ff",  # Azul
    wireframe=False
)
grandchild_node.set_position(3, 0, 0)  # Offset respecto al hijo

# Construcción de la jerarquía
child_node.add_child(grandchild_node)
parent_node.add_child(child_node)

print("✅ Jerarquía de 3 niveles creada:")
print(f"   Nivel 1: {parent_node.name} (Rojo)")
print(f"   └── Nivel 2: {child_node.name} (Verde)")
print(f"       └── Nivel 3: {grandchild_node.name} (Azul)")

✅ Jerarquía de 3 niveles creada:
   Nivel 1: Parent (Rojo)
   └── Nivel 2: Child (Verde)
       └── Nivel 3: Grandchild (Azul)


## Configuración de la Escena 3D

In [7]:
# Crear escena
scene = three.Scene()
scene.background = "#1a1a1a"  # Fondo oscuro

# Agregar jerarquía a la escena
scene.add(parent_node.get_three_object())

# Agregar ejes de referencia
axes = NodeFactory.create_axes_helper(10)
scene.add(axes)

# Iluminación
ambient_light = three.AmbientLight(color="#404040", intensity=0.5)
directional_light = three.DirectionalLight(color="#ffffff", intensity=0.8)
directional_light.position = [10, 10, 10]

scene.add(ambient_light)
scene.add(directional_light)

# Configurar cámara
camera = three.PerspectiveCamera(
    fov=45,
    aspect=1.5,
    near=0.1,
    far=1000,
    position=[15, 10, 15]
)
camera.lookAt([0, 0, 0])

# Controles de órbita
orbit_controls = three.OrbitControls(controlling=camera)

# Renderizador
renderer = three.Renderer(
    scene=scene,
    camera=camera,
    controls=[orbit_controls],
    width=900,
    height=600,
    antialias=True
)

print("✅ Escena 3D configurada correctamente")

✅ Escena 3D configurada correctamente


## Controles Interactivos (GUI)

Implementamos controles deslizantes para manipular las transformaciones en tiempo real.

In [ ]:
# Crear controles deslizantes para cada eje de rotación

# Funciones de callback para actualizar transformaciones en tiempo real
def update_parent_rotation_y(change):
    """Actualiza rotación Y del nodo padre"""
    parent_node.set_rotation(0, change['new'], 0)

def update_child_rotation_x(change):
    """Actualiza rotación X del nodo hijo"""
    child_rotation_y_value = child_rotation_y.value
    child_node.set_rotation(change['new'], child_rotation_y_value, 0)

def update_child_rotation_y(change):
    """Actualiza rotación Y del nodo hijo"""
    child_rotation_x_value = child_rotation_x.value
    child_node.set_rotation(child_rotation_x_value, change['new'], 0)

def update_grandchild_rotation_z(change):
    """Actualiza rotación Z del nodo nieto"""
    grandchild_rotation_y_value = grandchild_rotation_y.value
    grandchild_node.set_rotation(0, grandchild_rotation_y_value, change['new'])

def update_grandchild_rotation_y(change):
    """Actualiza rotación Y del nodo nieto"""
    grandchild_rotation_z_value = grandchild_rotation_z.value
    grandchild_node.set_rotation(0, change['new'], grandchild_rotation_z_value)

# Crear widgets de control
parent_rotation_y = FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Rot Y:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='500px')
)

child_rotation_x = FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Rot X:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='500px')
)

child_rotation_y = FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Rot Y:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='500px')
)

grandchild_rotation_z = FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Rot Z:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='500px')
)

grandchild_rotation_y = FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Rot Y:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width='500px')
)

# Vincular callbacks a los widgets
parent_rotation_y.observe(update_parent_rotation_y, names='value')
child_rotation_x.observe(update_child_rotation_x, names='value')
child_rotation_y.observe(update_child_rotation_y, names='value')
grandchild_rotation_z.observe(update_grandchild_rotation_z, names='value')
grandchild_rotation_y.observe(update_grandchild_rotation_y, names='value')

print("✅ Controles interactivos creados correctamente")

## Visualización Interactiva

Combina el renderizador 3D con los controles GUI.

In [10]:
# Panel de controles organizados
controls_panel = VBox([
    Label(value="🎮 Controles de Transformación", style={'font_weight': 'bold', 'font_size': '16px'}),
    Label(value="Nivel 1 - Padre (Rojo):"),
    parent_rotation_y,
    Label(value="Nivel 2 - Hijo (Verde):"),
    child_rotation_x,
    child_rotation_y,
    Label(value="Nivel 3 - Nieto (Azul):"),
    grandchild_rotation_z,
    grandchild_rotation_y,
])

# Layout final - Usar widgets.HTML en lugar de HTML de IPython.display
app = VBox([
    widgets.HTML("<h2 style='text-align:center;'>🌳 Árbol de Transformaciones Jerárquicas</h2>"),
    widgets.HTML("<p style='text-align:center;'>Usa los controles para ver cómo las transformaciones se propagan de padres a hijos</p>"),
    renderer,
    controls_panel
])

# Mostrar aplicación
display(app)

NameError: name 'parent_rotation_y' is not defined

## Animación Automática

Creamos una animación que demuestra las transformaciones encadenadas.

In [23]:
class AnimationController:
    """
    Controlador de animaciones para demostrar transformaciones jerárquicas.
    
    Implementa el patrón Strategy para diferentes tipos de animación.
    """
    
    def __init__(self, parent, child, grandchild):
        self.parent = parent
        self.child = child
        self.grandchild = grandchild
        self.time = 0
    
    def animate_hierarchy(self, duration: float = 10.0, fps: int = 30):
        """
        Anima la jerarquía completa demostrando transformaciones acumuladas.
        
        Args:
            duration: Duración total en segundos
            fps: Frames por segundo
        """
        frames = int(duration * fps)
        dt = duration / frames
        
        for i in range(frames):
            self.time += dt
            
            # Padre rota lentamente en Y
            parent_rot_y = self.time * 0.5
            self.parent.set_rotation(0, parent_rot_y, 0)
            
            # Hijo rota en X e Y (órbita + rotación propia)
            child_rot_x = self.time * 1.0
            child_rot_y = self.time * 0.8
            self.child.set_rotation(child_rot_x, child_rot_y, 0)
            
            # Nieto rota rápido en Z e Y
            grandchild_rot_z = self.time * 2.0
            grandchild_rot_y = self.time * 1.5
            self.grandchild.set_rotation(0, grandchild_rot_y, grandchild_rot_z)
            
            time.sleep(dt)
    
    def reset(self):
        """Reinicia todas las transformaciones"""
        self.time = 0
        self.parent.set_rotation(0, 0, 0)
        self.child.set_rotation(0, 0, 0)
        self.grandchild.set_rotation(0, 0, 0)


# Crear controlador de animación
animation_controller = AnimationController(
    parent_node,
    child_node,
    grandchild_node
)

print("✅ Controlador de animación creado")

✅ Controlador de animación creado


In [24]:
# Botón para iniciar animación
animate_button = widgets.Button(
    description='▶️ Iniciar Animación',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

reset_button = widgets.Button(
    description='🔄 Resetear',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)

def on_animate_click(b):
    animation_controller.animate_hierarchy(duration=10.0)

def on_reset_click(b):
    animation_controller.reset()
    parent_rotation_y.value = 0
    child_rotation_x.value = 0
    child_rotation_y.value = 0
    grandchild_rotation_z.value = 0
    grandchild_rotation_y.value = 0

animate_button.on_click(on_animate_click)
reset_button.on_click(on_reset_click)

animation_controls = HBox([animate_button, reset_button])
display(animation_controls)

TypeError: Euler.__init__() takes from 1 to 2 positional arguments but 5 were given

## Análisis de Transformaciones

Visualizamos matemáticamente cómo se acumulan las transformaciones.

In [25]:
def print_transformation_analysis():
    """
    Imprime un análisis detallado de las transformaciones actuales.
    """
    print("="*70)
    print("📊 ANÁLISIS DE TRANSFORMACIONES JERÁRQUICAS")
    print("="*70)
    
    # Padre
    print("\n🔴 NIVEL 1 - Nodo Padre (Rojo)")
    print(f"   Posición: {parent_node.local_transform['position']}")
    print(f"   Rotación: {parent_node.local_transform['rotation']}")
    print(f"   Espacio: Mundial (no tiene padre)")
    
    # Hijo
    print("\n🟢 NIVEL 2 - Nodo Hijo (Verde)")
    print(f"   Posición Local: {child_node.local_transform['position']}")
    print(f"   Rotación Local: {child_node.local_transform['rotation']}")
    print(f"   ⚠️  Hereda rotación del Padre: {parent_node.local_transform['rotation']}")
    print(f"   Espacio: Relativo al Padre")
    
    # Nieto
    print("\n🔵 NIVEL 3 - Nodo Nieto (Azul)")
    print(f"   Posición Local: {grandchild_node.local_transform['position']}")
    print(f"   Rotación Local: {grandchild_node.local_transform['rotation']}")
    print(f"   ⚠️  Hereda rotación del Hijo: {child_node.local_transform['rotation']}")
    print(f"   ⚠️  Hereda rotación del Padre: {parent_node.local_transform['rotation']}")
    print(f"   Espacio: Relativo al Hijo (que es relativo al Padre)")
    
    print("\n" + "="*70)
    print("💡 CONCLUSIÓN:")
    print("   Cuando el Padre rota → El Hijo y Nieto rotan con él")
    print("   Cuando el Hijo rota → El Nieto rota con él (pero no el Padre)")
    print("   Cuando el Nieto rota → Solo él rota (no afecta a ancestros)")
    print("="*70)

# Botón para análisis
analysis_button = widgets.Button(
    description='📊 Analizar Transformaciones',
    button_style='info',
    layout=widgets.Layout(width='250px', height='40px')
)

def on_analysis_click(b):
    print_transformation_analysis()

analysis_button.on_click(on_analysis_click)
display(analysis_button)

Button(button_style='info', description='📊 Analizar Transformaciones', layout=Layout(height='40px', width='250…

## Generación de GIF Animado

Capturamos frames de la animación para crear un GIF demostrativo.

In [ ]:
# Crear directorio de resultados
resultados_path = Path("resultados")
resultados_path.mkdir(exist_ok=True)

print("✅ Directorio de resultados creado")

In [ ]:
# Nota: La generación de GIF desde pythreejs requiere captura de pantalla manual
# o uso de herramientas externas como Selenium

print("""\n📝 INSTRUCCIONES PARA CAPTURA DE GIF:

1. Ejecuta la celda de animación (botón ▶️ Iniciar Animación)
2. Usa una herramienta de captura de pantalla (como QuickTime en Mac o OBS)
3. Graba la ventana del navegador durante la animación
4. Convierte el video a GIF usando una herramienta online o ffmpeg:
   
   ffmpeg -i input.mov -vf "fps=30,scale=800:-1:flags=lanczos" -c:v gif resultados/transformaciones_jerarquia.gif

Alternativamente, puedes usar el siguiente código para crear un GIF con matplotlib:
""")

# Método alternativo: Crear visualización con matplotlib
def create_matplotlib_animation():
    """
    Crea una animación alternativa usando matplotlib para generar el GIF.
    """
    from mpl_toolkits.mplot3d import Axes3D
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    frames = []
    n_frames = 60
    
    for i in range(n_frames):
        ax.clear()
        ax.set_xlim([-10, 10])
        ax.set_ylim([-10, 10])
        ax.set_zlim([-10, 10])
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Transformaciones Jerárquicas - Frame {i+1}/{n_frames}')
        
        t = i / n_frames * 2 * np.pi
        
        # Simular posiciones con transformaciones
        # Padre en el centro
        parent_pos = np.array([0, 0, 0])
        
        # Hijo orbita alrededor del padre
        child_offset = 4
        child_pos = parent_pos + np.array([
            child_offset * np.cos(t * 0.5),
            0,
            child_offset * np.sin(t * 0.5)
        ])
        
        # Nieto orbita alrededor del hijo
        grandchild_offset = 3
        grandchild_pos = child_pos + np.array([
            grandchild_offset * np.cos(t * 1.5),
            grandchild_offset * np.sin(t * 1.5),
            0
        ])
        
        # Dibujar cubos
        ax.scatter(*parent_pos, c='red', s=1000, marker='s', alpha=0.8, label='Padre')
        ax.scatter(*child_pos, c='green', s=700, marker='s', alpha=0.8, label='Hijo')
        ax.scatter(*grandchild_pos, c='blue', s=400, marker='s', alpha=0.8, label='Nieto')
        
        # Dibujar líneas de conexión
        ax.plot([parent_pos[0], child_pos[0]], 
                [parent_pos[1], child_pos[1]], 
                [parent_pos[2], child_pos[2]], 'k--', alpha=0.3)
        ax.plot([child_pos[0], grandchild_pos[0]], 
                [child_pos[1], grandchild_pos[1]], 
                [child_pos[2], grandchild_pos[2]], 'k--', alpha=0.3)
        
        ax.legend()
        
        # Capturar frame
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(image)
    
    plt.close()
    
    # Guardar GIF
    gif_path = resultados_path / "transformaciones_jerarquia.gif"
    imageio.mimsave(gif_path, frames, fps=20, loop=0)
    
    print(f"\n✅ GIF guardado en: {gif_path}")
    return gif_path

# Botón para generar GIF
gif_button = widgets.Button(
    description='🎬 Generar GIF',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

def on_gif_click(b):
    print("Generando GIF... (esto puede tomar unos segundos)")
    gif_path = create_matplotlib_animation()
    display(Image(filename=str(gif_path)))

gif_button.on_click(on_gif_click)
display(gif_button)

## Conclusiones y Aprendizajes

### 🎯 Conceptos Clave Demostrados

1. **Jerarquías de Transformación**:
   - Las transformaciones se propagan de padres a hijos de forma recursiva
   - Cada nodo mantiene transformaciones locales + hereda las de sus ancestros
   - La matriz de transformación final es el producto de todas las matrices ancestrales

2. **Espacios de Coordenadas**:
   - **Espacio Local**: Coordenadas relativas al padre inmediato
   - **Espacio Mundial**: Coordenadas absolutas en la escena
   - La conversión se hace multiplicando matrices de transformación

3. **Patrones de Diseño Aplicados**:
   - **Composite**: Para estructurar la jerarquía de nodos
   - **Factory**: Para encapsular creación de objetos complejos
   - **Strategy**: Para diferentes estrategias de animación

### 💡 Aplicaciones Prácticas

- **Animación de personajes**: Esqueletos con huesos jerárquicos
- **Robótica**: Cadenas cinemáticas de brazos robóticos
- **Sistemas planetarios**: Órbitas y rotaciones acumuladas
- **Interfaces 3D**: Menús y elementos UI anidados

### 🔬 Experimentos Sugeridos

1. Agregar un 4to nivel de jerarquía
2. Implementar diferentes tipos de transformaciones (escala, traslación)
3. Crear una cadena cinemática inversa (IK)
4. Añadir física/colisiones entre nodos

### 📚 Referencias

- [Three.js Documentation - Object3D](https://threejs.org/docs/#api/en/core/Object3D)
- [Computer Graphics: Principles and Practice](https://www.amazon.com/Computer-Graphics-Principles-Practice-3rd/dp/0321399528)
- [Transformation Matrices in Computer Graphics](https://en.wikipedia.org/wiki/Transformation_matrix)